## Preparing Datasets and Installs

In [1]:
import pandas as pd
import requests
import json
import os

import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
stock_list = pd.read_csv("data/equities.csv")
GICS = pd.read_excel("data/GICS.xlsx")

In [5]:
def clean_string(text):
    cleaned_text = text.replace('\xa0', ' ')  # Replace non-breaking space with a regular space
    # Add more cleaning steps as needed
    return cleaned_text

# Apply the cleaning function to the specified column
GICS['sub-industry'] = GICS['sub-industry'].apply(clean_string)

In [6]:
stock_list['symbol'].fillna("NaN", inplace=True)
stock_list.fillna(" ", inplace=True)

In [7]:
sub_industry = GICS[['sub_industry_code', 'sub-industry']].rename(columns={'sub_industry_code':'GicCode', 'sub-industry':'GicSubIndustry'}).astype(str)

In [8]:
other_industry = GICS.drop(columns=['sub_industry_code', 'sub-industry'])

In [9]:
stock_list['exchange'].unique()

array(['SHZ', 'KSC', 'HKG', 'KLS', 'KOE', 'BER', 'FRA', 'STU', ' ', 'ENX',
       'TWO', 'HAM', 'MUN', 'DUS', 'TAI', 'PAR', 'GER', 'IOB', 'LSE',
       'MEX', 'HAN', 'SAU', 'JPX', 'ASX', 'SAP', 'FKA', 'SES', 'TLO',
       'VIE', 'BSE', 'OSL', 'NSI', 'STO', 'MCE', 'MIL', 'BUD', 'SHH',
       'NYQ', 'SET', 'VAN', 'SAO', 'ISE', 'AMS', 'ATH', 'PNK', 'AQS',
       'CPH', 'TOR', 'NGM', 'NMS', 'NCM', 'ASE', 'CSE', 'JKT', 'HEL',
       'EBS', 'BUE', 'SGO', 'CNQ', 'NZE', 'CCS', 'JNB', 'BRU', 'TLV',
       'DOH', 'MCX', 'NEO', 'NYS', 'IST', 'NAS', 'LIS', 'LIT', 'PRA',
       'TAL', 'ICE', 'RIS', 'BTS', 'NSE', 'CAI', 'PCX', 'SAT', 'OBB',
       'NAE'], dtype=object)

In [10]:
def download_excel(exchange, output_folder):
    api_url = f"https://eodhistoricaldata.com/api/exchange-symbol-list/{exchange}?api_token=64e6bb917d6ca7.00493420"
    
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Check for any request errors
        
        content_disposition = response.headers.get('content-disposition')
        if content_disposition:
            filename = content_disposition.split('filename=')[1].replace('"', '')  # Remove double quotes
        else:
            filename = f"{exchange}_data.xlsx"  # Fallback filename
        
        output_path = os.path.join(output_folder, filename)
        
        with open(output_path, 'wb') as excel_file:
            excel_file.write(response.content)
        
        print(f"Excel file downloaded and saved as {filename} in {output_folder}.")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

download_folder = r"C:\Users\josia\Documents\GitHub\BT\BTWork\exchanges"


In [11]:
exchanges_full_list = ['US', 'LSE', 'AU', 'SHE','NSE', 'NEO','V', 'TO', 'BE', 'HM', 'XETRA', 'DU', 'MU', 'F', 'HA', 'STU', 'VI', 'PA', 'BR', 'MC', 'SW', 'LS', 'AS', 'ST', 'IR', 'CO', "OL", 'IC', 'HE',
      'XBOT', 'SEM', 'EGX', 'GSE', 'PR', 'BRVM', 'XNAI', 'BC', 'VFEX', 'MSE', 'XNSA', 'RSE', 'DSE', 'USE', 'LUSE', 'XZIM', 'TA', 'KQ', 'KO', 'BUD', 'WAR', 'PSE', 
      'JK', 'SHG', 'JSE', 'AT', 'SN', 'BK', 'KAR', 'SR', 'CM', 'VN', 'KLSE', 'BA', 'SA', 'RO', 'MX', 'IL', 'ZSE', 'MCX', 'TWO', 'TW', 'LIM', 'IS']


In [12]:
dataframes = {}
excluded = ['FUND', 'ETF', 'ETC', 'Mutual Fund', 'Note']
#exchanges_full_list = ['LU']
for i in exchanges_full_list:
    
    #download_excel(i, download_folder)
    file_name = f"{i}_LIST_OF_SYMBOLS.csv;"
    
    df = pd.read_csv(f"exchanges/{file_name}", dtype=str)    
    df = df[~df['Type'].isin(excluded)]
    dataframes[file_name] = df

In [13]:
total_rows = 0 
for i in exchanges_full_list:
    file_name = f"{i}_LIST_OF_SYMBOLS.csv;"
    #print(dataframes[file_name]['Exchange'].unique())
    total_rows = total_rows + len(dataframes[file_name])
    

## Creating Functions

In [14]:
def save_webpage_as_json(url, output_file, folder_name):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for any request errors
        
        content = response.text  # Get the content of the webpage
        
        # Parse content as JSON
        json_data = json.loads(content)
        
        subfolder_path = os.path.join('stocks', folder_name)
        os.makedirs(subfolder_path, exist_ok=True)
        
        output_path = os.path.join(subfolder_path, output_file)
        
        if not os.path.exists(output_path):
            with open(output_path, 'w', encoding='utf-8') as json_file:
                json.dump(json_data, json_file, indent=4)
            print(f"Webpage content saved as {output_file}")
            
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

In [15]:
def json_scrape(exchange, stock_list, exchange_code_official):
    for i in stock_list['Code']:
        string_start = "https://eodhistoricaldata.com/api/fundamentals/"
        string_end = "?api_token=64e6bb917d6ca7.00493420"
        url_string = string_start + i + "." + exchange + string_end
       
        output_filename = i + "_" + exchange + ".json"
        file_path = 'stocks/'+ exchange_code_official +"/"+output_filename
        if not os.path.exists(file_path):
            save_webpage_as_json(url_string, output_filename, exchange_code_official)
            

In [16]:
mydict = {}
mydict

{}

In [17]:
file_path = 'stocks/US/TSLA_US.json'
raw_data = open(file_path)
dat = json.load(raw_data)
dat['ESGScores']

if 'ESGScores' in dat:
    mydict['RatingDate'] = dat['ESGScores']['RatingDate']
    mydict['Adult'] = dat['ESGScores']['ActivitiesInvolvement']['0']['Involvement']
    mydict['Alcoholic'] = dat['ESGScores']['ActivitiesInvolvement']['1']['Involvement']
    mydict['AnimalTesting'] = dat['ESGScores']['ActivitiesInvolvement']['2']['Involvement'] 
    mydict['Catholic'] = dat['ESGScores']['ActivitiesInvolvement']['3']['Involvement']
    mydict['ControversialWeapons'] = dat['ESGScores']['ActivitiesInvolvement']['4']['Involvement']
    mydict['SmallArms'] = dat['ESGScores']['ActivitiesInvolvement']['5']['Involvement']
    mydict['FurLeather'] = dat['ESGScores']['ActivitiesInvolvement']['6']['Involvement']
    mydict['Gambling'] = dat['ESGScores']['ActivitiesInvolvement']['7']['Involvement']
    mydict['GMO'] = dat['ESGScores']['ActivitiesInvolvement']['8']['Involvement']
    mydict['MilitaryContract'] = dat['ESGScores']['ActivitiesInvolvement']['9']['Involvement']
    mydict['Nuclear'] = dat['ESGScores']['ActivitiesInvolvement']['10']['Involvement']
    mydict['Pesticides'] = dat['ESGScores']['ActivitiesInvolvement']['11']['Involvement']
    mydict['PalmOil'] = dat['ESGScores']['ActivitiesInvolvement']['12']['Involvement']
    mydict['Coal'] = dat['ESGScores']['ActivitiesInvolvement']['13']['Involvement']
    mydict['Tobacco'] = dat['ESGScores']['ActivitiesInvolvement']['14']['Involvement']
else:
    mydict['RatingDate'] = ''
    mydict['Adult'] = ''
    mydict['Alcoholic'] = ''
    mydict['AnimalTesting'] = ''
    mydict['Catholic'] = ''
    mydict['ControversialWeapons'] = ''
    mydict['SmallArms'] = ''
    mydict['FurLeather'] = ''
    mydict['Gambling'] = ''
    mydict['GMO'] = ''
    mydict['MilitaryContract'] =''
    mydict['Nuclear'] = ''
    mydict['Pesticides'] = ''
    mydict['PalmOil'] =''
    mydict['Coal'] = ''
    mydict['Tobacco'] = ''
    
mydict

{'RatingDate': '2019-01-01',
 'Adult': 'No',
 'Alcoholic': 'No',
 'AnimalTesting': 'No',
 'Catholic': 'No',
 'ControversialWeapons': 'No',
 'SmallArms': 'No',
 'FurLeather': 'No',
 'Gambling': 'No',
 'GMO': 'No',
 'MilitaryContract': 'No',
 'Nuclear': 'No',
 'Pesticides': 'No',
 'PalmOil': 'No',
 'Coal': 'No',
 'Tobacco': 'No'}

In [18]:
def stock_dataframe(exchange, stock_list, exchange_code_official):
    final_df = pd.DataFrame()
    for i in stock_list['Code']:
        
        output_filename = i + "_" + exchange + ".json"
        file_path = 'stocks/'+ exchange_code_official +"/"+output_filename
        if os.path.exists(file_path):
            raw_data = open(file_path)
            dict_data = json.load(raw_data)
   
            if len(dict_data) != 0 and 'GicSubIndustry' in dict_data['General']:
                
                new_dict = {'Code':dict_data['General']['Code'], 'Exchange':dict_data['General']['Exchange'], 'PrimaryTicker':dict_data['General']['PrimaryTicker'], 'Name':dict_data['General']['Name'], 'CountryName':dict_data['General']['CountryName'],
                            'ISIN':dict_data['General']['ISIN'], 'GicSector':dict_data['General']['GicSector'], 'GicGroup':dict_data['General']['GicGroup'],'GicIndustry':dict_data['General']['GicIndustry'],
                            'GicSubIndustry':dict_data['General']['GicSubIndustry'], 'Description':dict_data['General']['Description'], }
                #print(dict_data['General']['Code'])
#                 if 'ESGScores' in dict_data:
#                     if 'RatingDate' in dict_data['ESGScores']:
#                         new_dict['RatingDate'] = dict_data['ESGScores']['RatingDate']
#                     else:
#                         new_dict['RatingDate'] = ''
                    
#                     if 'Adult' in dict_data['ESGScores']:
#                         new_dict['Adult'] =  dict_data['ESGScores']['ActivitiesInvolvement']['0']['Involvement']
#                     else:
#                         new_dict['Adult'] = ''
                        
#                     if 'Alcoholic' in dict_data['ESGScores']:
#                         new_dict['Alcoholic'] = dict_data['ESGScores']['ActivitiesInvolvement']['1']['Involvement']
#                     else:
#                         new_dict['Alcoholic'] = ''
                        
#                     if 'AnimalTesting' in dict_data['ESGScores']:
#                         new_dict['AnimalTesting'] = dict_data['ESGScores']['ActivitiesInvolvement']['2']['Involvement'] 
#                     else:
#                         new_dict['AnimalTesting'] = ''
                        
#                     if 'Catholic' in dict_data['ESGScores']:
#                         new_dict['Catholic'] = dict_data['ESGScores']['ActivitiesInvolvement']['3']['Involvement']
#                     else:
#                         new_dict['Catholic'] = ''
                        
#                     if 'ControversialWeapons' in dict_data['ESGScores']:
#                         new_dict['ControversialWeapons'] = dict_data['ESGScores']['ActivitiesInvolvement']['4']['Involvement']
#                     else:
#                         new_dict['ControversialWeapons'] = ''
                        
#                     if 'SmallArms' in dict_data['ESGScores']:
#                         new_dict['SmallArms'] = dict_data['ESGScores']['ActivitiesInvolvement']['5']['Involvement']
#                     else:
#                         new_dict['SmallArms'] = ''
                        
#                     if 'FurLeather' in dict_data['ESGScores']:
#                         new_dict['FurLeather'] = dict_data['ESGScores']['ActivitiesInvolvement']['6']['Involvement']
#                     else:
#                         new_dict['FurLeather'] = ''
                        
#                     if 'Gambling' in dict_data['ESGScores']:
#                         new_dict['Gambling'] = dict_data['ESGScores']['ActivitiesInvolvement']['7']['Involvement']
#                     else:
#                         new_dict['Gambling'] = ''
                        
#                     if 'GMO' in dict_data['ESGScores']:
#                         new_dict['GMO'] = dict_data['ESGScores']['ActivitiesInvolvement']['8']['Involvement']
#                     else:
#                         new_dict['GMO'] = ''
                        
#                     if 'MilitaryContract' in dict_data['ESGScores']:
#                         new_dict['MilitaryContract'] = dict_data['ESGScores']['ActivitiesInvolvement']['9']['Involvement']
#                     else:
#                         new_dict['MilitaryContract'] = ''
                        
#                     if 'Nuclear' in dict_data['ESGScores']:
#                         new_dict['Nuclear'] = dict_data['ESGScores']['ActivitiesInvolvement']['10']['Involvement']
#                     else:
#                         new_dict['Nuclear'] = ''
                        
#                     if 'Pesticides' in dict_data['ESGScores']:
#                         new_dict['Pesticides'] = dict_data['ESGScores']['ActivitiesInvolvement']['11']['Involvement']
#                     else:
#                         new_dict['Pesticides'] = ''
                        
#                     if 'PalmOil' in dict_data['ESGScores']:
#                         new_dict['PalmOil'] = dict_data['ESGScores']['ActivitiesInvolvement']['12']['Involvement']
#                     else:
#                         new_dict['PalmOil'] = ''
                        
#                     if 'Coal' in dict_data['ESGScores']:
#                         new_dict['Coal'] = dict_data['ESGScores']['ActivitiesInvolvement']['13']['Involvement']
#                     else:
#                         new_dict['Coal'] = ''
                        
#                     if 'Tobacco' in dict_data['ESGScores']:
#                         new_dict['Tobacco'] = dict_data['ESGScores']['ActivitiesInvolvement']['14']['Involvement']
#                     else:
#                         new_dict['Tobacco'] = ''
                    
#                 # Added to test small big market cap
#                 if 'MarketCapitalization' in dict_data['Highlights']:
#                     new_dict['Market Cap'] = dict_data['Highlights']['MarketCapitalization']
#                 else:
#                     new_dict['Market Cap'] = ''
                       
                final_df = final_df.append(new_dict, ignore_index = True)
    return final_df

In [19]:
def formatting(exchange, stocks, ex_ticker):
    
    df = stock_dataframe(exchange, stocks, ex_ticker)
    df2 = pd.merge(df, sub_industry, on='GicSubIndustry', how='left') #.drop(columns=['GicSector', 'GicGroup', 'GicIndustry', 'GicSubIndustry'])
    df3 = pd.merge(df2, ex_df, on='Exchange', how='left')
    #cols = {'Code':'Stock Ticker', 'Name':'Company Name', 'GicCode':'GICSCode'}
    #df3.rename(columns=cols, inplace=True)
   # df3 = df3.reindex(columns=['Stock Ticker', 'Company Name', 'Exchange Ticker', 'Exchange Name', 'GICSCode', 'Market Cap'])
    return df3

## Stock codes for each exchange

In [20]:
ex_dict = {'Exchange':['AU','SHE','AS','AT','BA','BE','BK','BR','BUD','CM','CO','DSE','DU','F','HA','HE','HM','IC','IL','IR','IS','JK','JSE','KAR','KLSE',
                   'KO','KQ','LIM','LS','LSE','LUSE','MC','MCX','MSE','MU','MX','NEO','OL','PA','PR','PSE','RO','RSE','SA','SEM','SHG','SN','SR',
                   'ST','STU','SW','TA','TO','TW','TWO','V','VI','VN','WAR','XBOT','XETRA','XNAI','XNSA','ZSE', 'NSE',
                   'NYSE','NASDAQ','NYSE ARCA','NYSE MKT','OTCQX','OTCCE','OTCGREY','OTCMKTS','OTCQB'],
                   
         
         'Exchange Ticker': ['ASX','SHE','AMS','ATH','BUE','BER','BKK','BRU','BDP','COL','CPH','DSE','DUS','FRA','HAN','HEL','HAM','ISM','IL','DUB','IST','JKT',
                      'JSE','KAR','KLSE','KRX','KOSDAQ','LIM','ELI','LSE','LUSE','BME','MCX','MSE','MUN','BMV','NEO','OSL','EPA','PRG','PSE','BSE',
                      'RSE','BVMF','SEM','SHG','SSE','TASI','STO','STU','SWX','TLV','TSE','TPE','TWO','TSX','VIE','VSE','WSE','XBOT','XETRA','XNAI','XNSA','ZSE', 'NSE',
                      'NYSE', 'NASDAQ', 'NYSE ARCA', 'NYSE MKT', 'OTCQX', 'OTCCE','OTCGREY','OTCMKTS','OTCQB'], 
                 
         'Exchange Name':['Australia Securities Exchange', 'Shenzhen Stock Exchange', 'Euronext Amsterdam','Athens Exchange', 'Bolsa de Comercio de Buenos Aires', 'Boerse Berlin', 
                         'Stock Exchange of Thailand', 'Euronext Brussels', 'Budapest Stock Exchange', 'Colombo Stock Exchange', 'Nasdaq Copenhagen', 'Dar es Salaam Stock Exchange',
                          'Dusseldorf Stock Exchange', 'Frankfurt Stock Exchange','Hanover Stock Exchange', 'Nasdaq Helsinki','Hamburg Stock Exchange', 'Nasdaq Iceland', 
                          'London International Stock Exchange','Euronext Dublin', 'Borsa Istanbul', 'Indonesia Stock Exchange','Johannesbug Stock Exchange', 'Pakistan Stock Exchange',
                          'Bursa Malaysia', 'Korea Exchange', 'KOSDAQ', 'Bolsa De Valores De Lima','Euronext Lisbon','London Stock Exchange',
                          'Lusaka Stock Exchange', 'Madrid Exchange','Moscow Exchange', 'Malawi Stock Exchange', 'Munich Stock Exchange','Bolsa Mexicana Del Valores', 'NEO Exchange',
                          'Oslo Bors Asa','Euronext Paris','Prague Stock Exchange','Philippines Stock Exchange', 'Bursa de Valori Bucuresti', 'Rwanda Stock Exchange',
                          'B3 S.A. – Brasil, Bolsa, Balcão', 'Stock Exchange of Mauritius', 'Shanghai Stock Exchange', 'Bolsa de Santiago', 'Tadawul (Saudi Arabia)', 'Nasdaq Stockholm',
                          'Stuttgart Stock Exchange', 'SIX Swiss Exchange', 'Tel Aviv Stock Exchange', 'Toronto Stock Exchange','Taipei Exchange','Taiwan Stock Exchange','TSX Venture',
                          'Vienna Stock Exchange','Ho Chi Minh City Stock Exchange','Warsaw Stock Exchange', 'Botswanna Stock Exchange','Deutsche Boerse Xetra','Nairobi Stock Exchange',
                          'Nigeria Stock Exchange','Zagreb Stock Exchange', 'National Stock Exchange of India',
                          'New York Stock Exchage', 'NASDAQ', 'NYSE ARCA', 'NYSE MKT', 'OTCQX', 'OTCCE', 'OTCGREY', 'OTCMKTS', 'OTCBQ']}
                         
ex_df = pd.DataFrame(ex_dict)

In [21]:
excluded_exchange = ['PINK']
dataframes['US_LIST_OF_SYMBOLS.csv;'] = dataframes['US_LIST_OF_SYMBOLS.csv;'][~dataframes['US_LIST_OF_SYMBOLS.csv;']['Exchange'].isin(excluded_exchange)]

us_codes = dataframes['US_LIST_OF_SYMBOLS.csv;']
asx_codes = dataframes['AU_LIST_OF_SYMBOLS.csv;']  # Australia
she_codes = dataframes['SHE_LIST_OF_SYMBOLS.csv;'] # Shenzhen
nse_code = dataframes['NSE_LIST_OF_SYMBOLS.csv;'] # India
ams_codes = dataframes['AS_LIST_OF_SYMBOLS.csv;']  # Amsterdam
ath_codes = dataframes['AT_LIST_OF_SYMBOLS.csv;']  # Athens
bue_codes = dataframes['BA_LIST_OF_SYMBOLS.csv;']  # Buenos Aires
ber_codes = dataframes['BE_LIST_OF_SYMBOLS.csv;']  # Berlin
bkk_codes = dataframes['BK_LIST_OF_SYMBOLS.csv;']  # Bangkok
bru_codes = dataframes['BR_LIST_OF_SYMBOLS.csv;']  # Brussels
bdp_codes = dataframes['BUD_LIST_OF_SYMBOLS.csv;'] # Budapest
col_codes = dataframes['CM_LIST_OF_SYMBOLS.csv;']  # Sri Lanka / Colombo
cph_codes = dataframes['CO_LIST_OF_SYMBOLS.csv;']  # Denmark / Copenhagen
dse_codes = dataframes['DSE_LIST_OF_SYMBOLS.csv;'] # Tanzania / Dar es Salaam
dus_codes = dataframes['DU_LIST_OF_SYMBOLS.csv;']  # Dusseldorf
fra_codes = dataframes['F_LIST_OF_SYMBOLS.csv;']   # Frankfurt
han_codes = dataframes['HA_LIST_OF_SYMBOLS.csv;']  # Hanover
hel_codes = dataframes['HE_LIST_OF_SYMBOLS.csv;']  # Helsinki
ham_codes = dataframes['HM_LIST_OF_SYMBOLS.csv;']  # Hamburg
ice_codes = dataframes['IC_LIST_OF_SYMBOLS.csv;']  # Iceland
il_codes = dataframes['IL_LIST_OF_SYMBOLS.csv;']  # London International
dub_codes = dataframes['IR_LIST_OF_SYMBOLS.csv;']  # Ireland
ist_codes = dataframes['IS_LIST_OF_SYMBOLS.csv;']  # Istanbul
jkt_codes = dataframes['JK_LIST_OF_SYMBOLS.csv;']  # Jakarta
jse_codes = dataframes['JSE_LIST_OF_SYMBOLS.csv;'] #Johannesburg
kar_codes = dataframes['KAR_LIST_OF_SYMBOLS.csv;'] # Pakistan / Karachi
klse_codes = dataframes['KLSE_LIST_OF_SYMBOLS.csv;']# Malaysia
krx_codes = dataframes['KO_LIST_OF_SYMBOLS.csv;'] # Korea SE
kosdaq_codes = dataframes['KQ_LIST_OF_SYMBOLS.csv;'] #KOSDAQ
lim_codes = dataframes['LIM_LIST_OF_SYMBOLS.csv;'] # Lima
eli_codes = dataframes['LS_LIST_OF_SYMBOLS.csv;'] # Lisbon
lse_codes = dataframes['LSE_LIST_OF_SYMBOLS.csv;'] # London SE
#luxse_codes = dataframes['LU_LIST_OF_SYMBOLS.csv;'] # Luxembourg  - all funds (no GICS)
luse_codes = dataframes['LUSE_LIST_OF_SYMBOLS.csv;'] # Zambia / Lusaka
bme_codes = dataframes['MC_LIST_OF_SYMBOLS.csv;']  # Madrid / Bolsa de Madrid
mcx_codes = dataframes['MCX_LIST_OF_SYMBOLS.csv;'] # Russia
mse_codes = dataframes['MSE_LIST_OF_SYMBOLS.csv;'] # Malawi SE
mun_codes = dataframes['MU_LIST_OF_SYMBOLS.csv;'] # Munich
bmv_codes = dataframes['MX_LIST_OF_SYMBOLS.csv;'] # Bolsa Mexicana De Valores
neo_codes = dataframes['NEO_LIST_OF_SYMBOLS.csv;'] # Canada
osl_codes = dataframes['OL_LIST_OF_SYMBOLS.csv;'] # Norway / Oslo
epa_codes = dataframes['PA_LIST_OF_SYMBOLS.csv;'] # Paris
prg_codes = dataframes['PR_LIST_OF_SYMBOLS.csv;'] # Prague
pse_codes = dataframes['PSE_LIST_OF_SYMBOLS.csv;'] # Philippines
bse_codes = dataframes['RO_LIST_OF_SYMBOLS.csv;'] # Romania / Bucharest
rse_codes = dataframes['RSE_LIST_OF_SYMBOLS.csv;'] # Rwanda
bvmf_codes = dataframes['SA_LIST_OF_SYMBOLS.csv;'] # Sao Paulo
sem_codes = dataframes['SEM_LIST_OF_SYMBOLS.csv;'] # SE Mauritius
shg_codes = dataframes['SHG_LIST_OF_SYMBOLS.csv;'] # Shanghai
sse_codes = dataframes['SN_LIST_OF_SYMBOLS.csv;'] # Chile / santiago
tasi_codes = dataframes['SR_LIST_OF_SYMBOLS.csv;'] # Saudi arabia
sto_codes = dataframes['ST_LIST_OF_SYMBOLS.csv;'] # Stockholm
stu_codes = dataframes['STU_LIST_OF_SYMBOLS.csv;'] # Stuttgart
swx_codes = dataframes['SW_LIST_OF_SYMBOLS.csv;'] # Switzerland
tlv_codes = dataframes['TA_LIST_OF_SYMBOLS.csv;'] # Israel
tse_codes = dataframes['TO_LIST_OF_SYMBOLS.csv;'] # Toronto
tpe_codes = dataframes['TW_LIST_OF_SYMBOLS.csv;'] # Taipei
two_codes = dataframes['TWO_LIST_OF_SYMBOLS.csv;'] # Taiwan
tsx_codes = dataframes['V_LIST_OF_SYMBOLS.csv;'] # TSX Venture
vie_codes = dataframes['VI_LIST_OF_SYMBOLS.csv;'] # Vienna
vse_codes = dataframes['VN_LIST_OF_SYMBOLS.csv;'] # Ho chi minh
wse_codes = dataframes['WAR_LIST_OF_SYMBOLS.csv;'] # Warsaw
xbot_codes = dataframes['XBOT_LIST_OF_SYMBOLS.csv;'] # Botswanna
xetra_codes = dataframes['XETRA_LIST_OF_SYMBOLS.csv;'] # Deutche Boerse Xetra
xnai_codes = dataframes['XNAI_LIST_OF_SYMBOLS.csv;'] # Kenya
xnsa_codes = dataframes['XNSA_LIST_OF_SYMBOLS.csv;'] # Nigeria
zse_codes = dataframes['ZSE_LIST_OF_SYMBOLS.csv;'] # Zagreb
nse_codes = dataframes['NSE_LIST_OF_SYMBOLS.csv;'] # India

In [22]:
tse_codes['Code'].fillna("NA", inplace=True)
us_codes['Exchange'].fillna("US", inplace=True)
us_codes['Code'].fillna("NAN", inplace=True)

## Scraping

In [23]:
json_scrape("AU", asx_codes, "ASX")
json_scrape("SHE", she_codes, "SHE")
json_scrape('AS', ams_codes, 'AMS')
json_scrape('AT', ath_codes, 'ATH')
json_scrape('BA', bue_codes, 'BUE')
json_scrape('BE', ber_codes, 'BER')
json_scrape('BK', bkk_codes, 'BKK')
json_scrape('BR', bru_codes, 'BRU')
json_scrape('BUD', bdp_codes, 'BDP')
json_scrape('CM', col_codes, 'COL')
json_scrape('CO', cph_codes, 'CPH')
json_scrape('DSE', dse_codes, 'DSE')
json_scrape('DU', dus_codes, 'DUS')
json_scrape('F', fra_codes, 'FRA')
json_scrape('HA', han_codes, 'HAN')
json_scrape('HE', hel_codes, 'HEL')
json_scrape('HM', ham_codes, 'HAM')
json_scrape('IC', ice_codes, 'ICE')
json_scrape('IL', il_codes, 'IL')
json_scrape('IR', dub_codes, 'DUB')
json_scrape('IS', ist_codes, 'IST')
json_scrape('JK', jkt_codes, 'JKT')
json_scrape('JSE', jse_codes, 'JSE')
json_scrape('KAR', kar_codes, 'KAR')
json_scrape('KLSE', klse_codes, 'KLSE')
json_scrape('KO', krx_codes, 'KRX')
json_scrape('KQ', kosdaq_codes, 'KOSDAQ')
json_scrape('LIM', lim_codes, 'LIM')
json_scrape('LS', eli_codes, 'ELI')
json_scrape('LSE', lse_codes, 'LSE')
json_scrape('LUSE', luse_codes, 'LUSE')
json_scrape('MC', bme_codes, 'BME')
json_scrape('MC', bme_codes, 'BME')
json_scrape('MCX', mcx_codes, 'MCX')
json_scrape('MSE', mse_codes, 'MSE')
json_scrape('MU', mun_codes, 'MUN')
json_scrape('MX', bmv_codes, 'BMV')
json_scrape('NEO', neo_codes, 'NEO')
json_scrape('OL', osl_codes, 'OSL')
json_scrape('PA', epa_codes, 'EPA')
json_scrape('PR', prg_codes, 'PRG')
json_scrape('PSE', pse_codes, 'PSE')
json_scrape('RO', bse_codes, 'BSE')
json_scrape('RSE', rse_codes, 'RSE')
json_scrape('SA', bvmf_codes, 'BVMF')
json_scrape('SEM', sem_codes, 'SEM')
json_scrape('SHG', shg_codes, 'SHG')
json_scrape('SN', sse_codes, 'SSE')
json_scrape('SR', tasi_codes, 'TASI')
json_scrape('ST', sto_codes, 'STO')
json_scrape('STU', stu_codes, 'STU')
json_scrape('SW', swx_codes, 'SWX')
json_scrape('TA', tlv_codes, 'TLV')
json_scrape('TO', tse_codes, 'TSE')
json_scrape('TW', tpe_codes, 'TPE')
json_scrape('TWO', two_codes, 'TWO')
json_scrape('V', tsx_codes, 'TSX')
json_scrape('VI', vie_codes, 'VIE')
json_scrape('VN', vse_codes, 'VSE')
json_scrape('WAR', wse_codes, 'WSE')
json_scrape('XBOT', xbot_codes, 'XBOT')
json_scrape('XETRA', xetra_codes, 'XETRA')
json_scrape('XNAI', xnai_codes, 'XNAI')
json_scrape('XNSA', xnsa_codes, 'XNSA')
json_scrape('ZSE', zse_codes, 'ZSE')
json_scrape('NSE', nse_codes, 'NSE')
json_scrape('US', us_codes, "US")

## GICS Merging

In [24]:
# dealing with weird US stuff

usa_unf = stock_dataframe('US', us_codes, 'US')

usa_drop = ['AMEX','BATS', 'US', 'NMFQS']
usa = usa_unf[~usa_unf['Exchange'].isin(usa_drop)]



In [25]:
usa2 = pd.merge(usa, sub_industry, on='GicSubIndustry', how='left') #.drop(columns=['GicSector', 'GicGroup', 'GicIndustry', 'GicSubIndustry'])
usa3 = pd.merge(usa2, ex_df, on='Exchange', how='left')
#cols = {'Code':'Stock Ticker', 'Name':'Company Name', 'GicCode':'GICSCode'}
#usa3.rename(columns=cols, inplace=True)
#usa3 = usa3.reindex(columns=['Stock Ticker', 'Company Name', 'Exchange Ticker', 'Exchange Name', 'GICSCode', 'Market Cap'])


In [26]:
usa4 = pd.merge(usa2, ex_df, on='Exchange', how='left')

In [27]:
asx = formatting('AU', asx_codes, 'ASX')
ams = formatting('AS', ams_codes, 'AMS')
ath = formatting('AT', ath_codes, 'ATH')
bue = formatting('BA', bue_codes, 'BUE')
ber = formatting('BE', ber_codes, 'BER')
bkk = formatting('BK', bkk_codes, 'BKK')
bru = formatting('BR', bru_codes, 'BRU')
bdp = formatting('BUD', bdp_codes, 'BDP')
she = formatting("SHE", she_codes, "SHE")
col = formatting('CM', col_codes, 'COL')
cph = formatting('CO', cph_codes, 'CPH')
dse = formatting('DSE', dse_codes, 'DSE')
dus = formatting('DU', dus_codes, 'DUS')
fra = formatting('F', fra_codes, 'FRA')
han = formatting('HA', han_codes, 'HAN')
hel = formatting('HE', hel_codes, 'HEL')
ham = formatting('HM', ham_codes, 'HAM')
ice = formatting('IC', ice_codes, 'ICE')
il = formatting('IL', il_codes, 'IL')
dub = formatting('IR', dub_codes, 'DUB')
ist = formatting('IS', ist_codes, 'IST')
jkt = formatting('JK', jkt_codes, 'JKT')
jse = formatting('JSE', jse_codes, 'JSE')
kar = formatting('KAR', kar_codes, 'KAR')
klse = formatting('KLSE', klse_codes, 'KLSE')
krx = formatting('KO', krx_codes, 'KRX')
kosdaq = formatting('KQ', kosdaq_codes, 'KOSDAQ')
lim = formatting('LIM', lim_codes, 'LIM')
eli = formatting('LS', eli_codes, 'ELI')
lse = formatting('LSE', lse_codes, 'LSE')

luse = formatting('LUSE', luse_codes, 'LUSE')
bme = formatting('MC', bme_codes, 'BME')
mcx = formatting('MCX', mcx_codes, 'MCX')
mse = formatting('MSE', mse_codes, 'MSE')
mun = formatting('MU', mun_codes, 'MUN')
bmv = formatting('MX', bmv_codes, 'BMV')
neo = formatting('NEO', neo_codes, 'NEO')
osl = formatting('OL', osl_codes, 'OSL')
epa = formatting('PA', epa_codes, 'EPA')
prg = formatting('PR', prg_codes, 'PRG')
pse = formatting('PSE', pse_codes, 'PSE')
bse = formatting('RO', bse_codes, 'BSE')
rse = formatting('RSE', rse_codes, 'RSE')
bvmf = formatting('SA', bvmf_codes, 'BVMF')
sem = formatting('SEM', sem_codes, 'SEM')
shg = formatting('SHG', shg_codes, 'SHG')
sse = formatting('SN', sse_codes, 'SSE')
tasi = formatting('SR', tasi_codes, 'TASI')
sto = formatting('ST', sto_codes, 'STO')
stu = formatting('STU', stu_codes, 'STU')
swx = formatting('SW', swx_codes, 'SWX')
tlv = formatting('TA', tlv_codes, 'TLV')
tse = formatting('TO', tse_codes, 'TSE')
tpe = formatting('TW', tpe_codes, 'TPE')
two = formatting('TWO', two_codes, 'TWO')
tsx = formatting('V', tsx_codes, 'TSX')
vie = formatting('VI', vie_codes, 'VIE')
vse = formatting('VN', vse_codes, 'VSE')
wse = formatting('WAR', wse_codes, 'WSE')
xbot = formatting('XBOT', xbot_codes, 'XBOT')
xetra = formatting('XETRA', xetra_codes, 'XETRA')
xnai = formatting('XNAI', xnai_codes, 'XNAI')
xnsa = formatting('XNSA', xnsa_codes, 'XNSA')
zse = formatting('ZSE', zse_codes, 'ZSE')
nse = formatting('NSE', nse_codes, 'NSE')



In [28]:
# dealing with weird exchange notations in each STO and CPH

sto['Exchange Ticker'] = 'STO'
sto['Exchange Name'] = 'Nasdaq Stockholm'

cph['Exchange Ticker'] = 'CPH'
cph['Exchange Name'] = 'Nasdaq Copenhagen'

In [29]:
# dictionary of all exchanges except US
exch = {'ASX':asx,'SHE':she,'AMS':ams,'ATH':ath,'BUE':bue,'BER':ber,'BKK':bkk,'BRU':bru,'BDP':bdp,'COL':col,'CPH':cph,'DSE':dse,'DUS':dus,'FRA':fra,'HAN':han,
        'HEL':hel,'HAM':ham,'ICE':ice,'IL':il,'DUB':dub,'IST':ist,'JKT':jkt, 'JSE':jse,'KAR':kar,'KLSE':klse,'KRX':krx,'KOSDAQ':kosdaq,'LIM':lim,'ELI':eli,
        'LSE':lse,'LUSE':luse,'BME':bme,'MCX':mcx,'MSE':mse,'MUN':mun,'BMV':bmv,'NEO':neo,'OSL':osl,'EPA':epa,'PRG':prg,'PSE':pse,'BSE':bse,'RSE':rse,'BVMF':bvmf,
        'SEM':sem,'SHG':shg,'SSE':sse,'TASI':tasi,'STO':sto,'STU':stu,'SWX':swx,'TLV':tlv,'TSE':tse,'TPE':tpe,'TWO':two,'TSX':tsx,'VIE':vie,'VSE':vse,'WSE':wse,'XBOT':xbot,'XETRA':xetra,
        'XNAI':xnai,'XNSA':xnsa,'ZSE':zse, 'NSE':nse}

In [30]:
# putting all dataframes into a list

stock_gic = []
for index, row in ex_df.iterrows():
    df_name = row['Exchange Ticker']
    if df_name in exch:
        stock_gic.append(exch[df_name])

In [31]:
# combining all exchanges except US

stock_gic_df = pd.concat(stock_gic, ignore_index=True) #.sort_values(by='Exchange Ticker',ascending=True)

In [32]:
# adding US Stocks cause tricky

stock_gic_df2 = pd.concat([stock_gic_df, usa4], axis=0).reset_index(drop=True)

In [33]:
stock_gic_df2.describe()

,Code,Exchange,PrimaryTicker,Name,CountryName,ISIN,GicSector,GicGroup,GicIndustry,GicSubIndustry,Description,GicCode,Exchange Ticker,Exchange Name
count,71188,71185,37167,71188,71188,47524,62233,62232,62233,62132,61965,61064,71185,71185
unique,49851,75,27661,57800,50,36417,11,26,72,165,45805,156,73,73
top,HAL,F,AAPL.US,Bank of America Corporation,Germany,US0378331005,Materials,Materials,Metals & Mining,Diversified Metals & Mining,NA,15104020,FRA,Frankfurt Stock Exchange
freq,15,12275,16,24,26667,16,18058,18058,15370,13400,71,13400,12275,12275


In [34]:
GICS = GICS.rename(columns = {'sector_code':'SectorCode','sector':'GicSector', 'industry_group_code': 'IndustryGroupCode', 
                              'industry_group':'GicIndustryGroup', 'industry_code':'IndustryCode', 'industry':'GicIndustry', 
                              'sub_industry_code':'SubIndustryCode', 'sub-industry':'GicSubIndustry'})

In [35]:
GICS.head()

,SectorCode,GicSector,IndustryGroupCode,GicIndustryGroup,IndustryCode,GicIndustry,SubIndustryCode,GicSubIndustry
0,10,Energy,1010,Energy,101010,Energy Equipment & Services,10101010,Oil & Gas Drilling
1,10,Energy,1010,Energy,101010,Energy Equipment & Services,10101020,Oil & Gas Equipment & Services
2,10,Energy,1010,Energy,101020,"Oil, Gas & Consumable Fuels",10102010,Integrated Oil & Gas
3,10,Energy,1010,Energy,101020,"Oil, Gas & Consumable Fuels",10102020,Oil & Gas Exploration & Production
4,10,Energy,1010,Energy,101020,"Oil, Gas & Consumable Fuels",10102030,Oil & Gas Refining & Marketing


In [119]:
dataframe = pd.merge(stock_gic_df2, GICS, on='GicSubIndustry') 

In [120]:
dataframe['PrimaryTicker'] = dataframe['PrimaryTicker'].str.split(".").str[1]

In [121]:
dataframe.loc[dataframe['PrimaryTicker'] == 'NZ', 'PrimaryTicker'] = 'AU'
dataframe.loc[dataframe['Exchange'] == 'NASDAQ', 'Exchange'] = 'US'
dataframe.loc[dataframe['Exchange'] == 'NYSE MKT', 'Exchange'] = 'US'
dataframe.loc[dataframe['Exchange'] == 'OTCGREY', 'Exchange'] = 'US'
dataframe.loc[dataframe['Exchange'] == 'OTCQB', 'Exchange'] = 'US'
dataframe.loc[dataframe['Exchange'] == 'OTCQX', 'Exchange'] = 'US'
dataframe.loc[dataframe['Exchange'] == 'OTCMKTS', 'Exchange'] = 'US'
dataframe.loc[dataframe['Exchange'] == 'NYSE ARCA', 'Exchange'] = 'US'
dataframe.loc[dataframe['Exchange'] == 'OTCCE', 'Exchange'] = 'US'
dataframe.loc[dataframe['Exchange'] == 'NYSE', 'Exchange'] = 'US'

In [122]:
for idx in dataframe.index:
    if dataframe['PrimaryTicker'].loc[idx] is not None:
        if dataframe['PrimaryTicker'].loc[idx] != dataframe['Exchange'].loc[idx]:
            dataframe.drop(idx, axis='index', inplace=True)

In [128]:
dataframe.head(10)

,Stock Ticker,Company Name,Exchange Ticker,Exchange Name,PrimaryTicker,CountryName,ISIN,Description,SectorCode,Sector,...,SmallArms,FurLeather,Gambling,GMO,MilitaryContract,Nuclear,Pesticides,PalmOil,Coal,Tobacco
0,14D,1414 Degrees Ltd,ASX,Australia Securities Exchange,NaN,Australia,AU0000010696,1414 Degrees Limited offers thermal energy sto...,55,Utilities,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CCE,Carnegie Clean Energy Ltd,ASX,Australia Securities Exchange,NaN,Australia,AU000000CCE7,Carnegie Clean Energy Limited develops and com...,55,Utilities,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GNE,Genesis Energy Ltd,ASX,Australia Securities Exchange,NaN,Australia,NZGNEE0001S7,"Genesis Energy Limited generates, trades in, a...",55,Utilities,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VPR,Volt Power Group Ltd,ASX,Australia Securities Exchange,NaN,Australia,AU000000VPR1,"Volt Power Group Limited, together with its su...",55,Utilities,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000027,Shenzhen Energy Group Co Ltd,SHE,Shenzhen Stock Exchange,NaN,China,CNE000000933,"Shenzhen Energy Group Co., Ltd. engages in the...",55,Utilities,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,000531,Guangzhou Hengyun Enterprises Holding Ltd,SHE,Shenzhen Stock Exchange,NaN,China,CNE000000DS2,Guangzhou Hengyun Enterprises Holding Ltd gene...,55,Utilities,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,000543,An Hui Wenergy Co Ltd,SHE,Shenzhen Stock Exchange,NaN,China,CNE000000DF9,An Hui Wenergy Company Limited engages in the ...,55,Utilities,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,000600,Jointo Energy Investment Co Ltd Hebei,SHE,Shenzhen Stock Exchange,NaN,China,CNE000000FT5,"Jointo Energy Investment Co., Ltd. Hebei inves...",55,Utilities,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,000601,Guangdong Shaoneng Group Co Ltd,SHE,Shenzhen Stock Exchange,NaN,China,CNE0000009H8,"GuangDong ShaoNeng Group Co., Ltd. engages in ...",55,Utilities,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,000690,Guangdong Baolihua New Energy Stock Co LTD,SHE,Shenzhen Stock Exchange,NaN,China,CNE000000P12,"Guangdong Baolihua New Energy Stock Co., Ltd. ...",55,Utilities,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
dataframe.drop(columns=['Exchange','PrimaryTicker', 'GicSector_x', 'GicGroup', 'GicIndustry_x', 'GicCode'], inplace=True)
dcols = {'Code':'Stock Ticker', 'Name':'Company Name', 'GicSector_y':'Sector', 'GicSubIndustry':'SubIndustry', 'GicIndustry_y':'Industry', 'GicIndustryGroup':'IndustryGroup'}
dataframe.rename(columns=dcols, inplace=True)
dataframe = dataframe.reindex(columns=['Stock Ticker', 'Company Name', 'Exchange Ticker', 'Exchange Name', 'PrimaryTicker', 'CountryName', 'ISIN', 'Description', 'SectorCode', 'Sector', 'IndustryGroupCode', 
                                       'IndustryGroup', 'IndustryCode', 'Industry', 'SubIndustryCode', 'SubIndustry', 'RatingDate', 'Adult', 'Alcoholic', 'AnimalTesting', 'Catholic', 'ControversialWeapons',
                                       'SmallArms', 'FurLeather', 'Gambling', 'GMO', 'MilitaryContract', 'Nuclear', 'Pesticides', 'PalmOil', 'Coal', 'Tobacco'])


In [127]:
dataframe['Exchange Ticker'].unique()

array(['ASX', 'SHE', 'ATH', 'BUE', 'BKK', 'FRA', 'IST', 'JKT', 'KOSDAQ',
       'LSE', 'MCX', 'OSL', 'BVMF', 'SHG', 'SSE', 'STU', 'TLV', 'TSE',
       'TPE', 'WSE', 'XETRA', 'NSE', 'NYSE', 'OTCCE', 'AMS', 'BRU', 'CPH',
       'HEL', 'IL', 'DUB', 'KLSE', 'KRX', 'BME', 'MUN', 'BMV', 'NEO',
       'EPA', 'STO', 'SWX', 'TWO', 'TSX', 'VIE', 'NASDAQ', 'NYSE MKT',
       'OTCGREY', 'OTCQB', 'OTCQX', 'OTCMKTS', 'BER', 'BDP', 'DUS', 'HAM',
       'JSE', 'KAR', 'PRG', 'TASI', 'PSE', 'HAN', 'NYSE ARCA', 'ELI',
       'ZSE'], dtype=object)

In [125]:
dataframe.to_csv('data/CompiledStock.csv', index=False)